In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from scipy.stats import t
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.utils import shuffle
import statistics as stat

import warnings 
warnings.filterwarnings("ignore")

# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Содержание
1. [Загрузка и подготовка данных](#0),
2. [Обучение и проверка модели](#1),
3. [Подготовка к расчёту прибыли](#2),
4. [асчёт прибыли и рисков](#3),
5. [Вывод](#4),

# 1. Загрузка и подготовка данных <a id='0'></a>

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(df_0.info())
print(df_1.info())
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [3]:
display(df_0.head())
display(df_1.head())
display(df_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Пропусков в данных нет. Удалю столбец id во всех датасетах, так как признак id не важен для работы модели.

In [4]:
df_0 = df_0.drop('id', axis = 1)
df_1 = df_1.drop('id', axis = 1)
df_2 = df_2.drop('id', axis = 1)

# 2. Обучение и проверка модели <a id='1'></a>

In [5]:
model = LinearRegression()
def model_log(df):     
    target = df['product']
    features = df.drop('product', axis = 1)
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size = 0.25, random_state = 12345)
    
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    mean = df['product'].mean()
    r2 = r2_score(target_valid, predicted_valid)
    rmse = (mean_squared_error(target_valid, predicted_valid))**0.5
    cvs = stat.mean(cross_val_score(model, features, target, cv = 5))
    mae = mean_absolute_error(target_valid, predicted_valid)
    print('Средний запас в регионе:', round(mean, 2))
    print('Средняя оценка качества модели:',cvs)
    print('rmse:', rmse)
    print('r2:', r2)
    print('mae:', mae)
    
    return target_valid, predicted_valid, mean, rmse, r2

In [6]:
print('Регион-0')
target_valid_0, predicted_valid_0, mean_0, rmse_0, r2_0 = model_log(df_0)

Регион-0
Средний запас в регионе: 92.5
Средняя оценка качества модели: 0.2754913072690447
rmse: 37.5794217150813
r2: 0.27994321524487786
mae: 30.919600777151313


In [7]:
print('Регион-1')
target_valid_1, predicted_valid_1, mean_1, rmse_1, r2_1 = model_log(df_1)

Регион-1
Средний запас в регионе: 68.83
Средняя оценка качества модели: 0.9996243728923551
rmse: 0.893099286775616
r2: 0.9996233978805127
mae: 0.718766244212475


In [8]:
print('Регион-2')
target_valid_2, predicted_valid_2, mean_2, rmse_2, r2_2 = model_log(df_2)

Регион-2
Средний запас в регионе: 95.0
Средняя оценка качества модели: 0.1987156246205129
rmse: 40.02970873393434
r2: 0.20524758386040443
mae: 32.792652105481814


Модель в регионе-1 показала лучший результат по метрикам, однако запас сырья в регионе меньше чем в других.

# 3. Подготовка к расчёту прибыли <a id='2'></a>

In [9]:
#необходимые переменные
dots = 500 #количество точек для иследования
budget = 10**10 #бюджет проекта
one_well_price = 5 * 10**7 #стоимость одной скважины
wells_count = budget / one_well_price #количество скважин
one_barrel_profit = 450 #прибыль с одного барреля
th_barrel_profit = one_barrel_profit * 1000 #прибыль с 1 тысячи бареллей

In [10]:
#рассчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки.
min_mean_product = (budget / th_barrel_profit) / wells_count
min_mean_product

111.11111111111111

Исходя из полученных данных, можно сделать вывод, что необходимо минимум в среднем 111.11 тысяч баррелей нефти в месторождениях.

In [11]:
#функция для расчёта прибыли по набору отобранных месторождений и предсказаний модели.
def profit(target_valid, predicted_valid):
    predicted_valid_sort = pd.Series(predicted_valid)
    predicted_valid_sort = predicted_valid_sort.sort_values(ascending = False)
    target_valid = target_valid.reset_index(drop = True)
    temp = target_valid[predicted_valid_sort.index][:200]
    return temp.sum() * th_barrel_profit - budget

In [12]:
df_0_profit = profit(target_valid_0, predicted_valid_0)
print('Операционная прибыль в регионе-0:', df_0_profit / 1000000000)

Операционная прибыль в регионе-0: 3.3208260431398524


In [13]:
df_1_profit = profit(target_valid_1, predicted_valid_1)
print('Операционная прибыль в регионе-1:', df_1_profit / 1000000000)

Операционная прибыль в регионе-1: 2.4150866966815108


In [14]:
df_2_profit = profit(target_valid_2, predicted_valid_2)
print('Операционная прибыль в регионе-2:', df_2_profit / 1000000000)

Операционная прибыль в регионе-2: 2.7103499635998327


По 200 скважин, самая большая прибыль прогнозирыется в регионе-0 (3.32 млрд.), на втором месте идёт регион-2 (2.71 млрд.) и на последнем регион-1 с прибылью 2.41 млрд.

# 4. Расчёт прибыли и рисков  <a id='3'></a>

In [15]:
state = np.random.RandomState(12345)

#функция для bootstrap и нахождения средней прибыли, 95% интервала и риска убытков.
def bootstrap(target_valid, predicted_valid):    
    values = []
    for i in range(1000):
        target_valid = target_valid.reset_index(drop = True)
        target_subsample = target_valid.sample(n = dots, replace = True, random_state = state)
        predicted_subsample = predicted_valid[target_subsample.index]
        values.append(profit(target_subsample, predicted_subsample))
    
    values = pd.Series(values)
    mean = values.mean()
    #confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem()) #дов. интервал
    lower = values.quantile(q = 0.025)
    upper = values.quantile(q = 0.975)
    risk_of_loss = st.percentileofscore(values, 0) #риск убытков
        
    return values, mean, lower, upper, risk_of_loss

In [16]:
values_0, mean_0, lower_0, upper_0, risk_of_loss_0 = bootstrap(target_valid_0, predicted_valid_0)
values_1, mean_1, lower_1, upper_1, risk_of_loss_1 = bootstrap(target_valid_1, predicted_valid_1)
values_2, mean_2, lower_2, upper_2, risk_of_loss_2 = bootstrap(target_valid_2, predicted_valid_2)

In [17]:
#сравню среднюю прибыль в тыс.
print('Регион-0:', mean_0 / 1000)
print('Регион-1:', mean_1 / 1000)
print('Регион-2:', mean_2 / 1000)

Регион-0: 396164.98480237107
Регион-1: 461155.81727723975
Регион-2: 392950.4751706045


In [18]:
#сравню доверительные интервалы
print('Регион-0:', lower_0, upper_0)
print('Регион-1:', lower_1, upper_1)
print('Регион-2:', lower_2, upper_2)

Регион-0: -111215545.89049526 909766941.5534225
Регион-1: 78050810.7517417 862952060.2637235
Регион-2: -112227625.37857565 934562914.5511636


In [19]:
#риски убытков
print('Регион-0:', risk_of_loss_0)
print('Регион-1:', risk_of_loss_1)
print('Регион-2:', risk_of_loss_2)

Регион-0: 6.9
Регион-1: 0.7
Регион-2: 6.5


# Вывод <a id='4'></a>

Вероятность убытков меньше 2.5% только в регионе-1, самое высокое среднее и лучший доверительный интервал 95% показал регион-1. Исходя из этого рекомендуемым регионом к разарботке стоит выбрать 'регион-1'.

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован